In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
# parmeters
grid_x, grid_y = 300, 200
I = 1.0  # slope gradient
beta = 0.05
Dh = 10  # erosion height
r = 200  # number of raindrops
total_drops = 5000

# Initialize the height map
height_map = np.zeros((grid_x, grid_y))
for i in range(grid_y):
    height_map[:, i] = I * i + 1e-6 * np.random.rand(grid_x)  # small randomnes to simulate real terrain

In [ ]:
def get_neighbors(x, y, width, height):
    neighbors = []

    # Left cell
    if x - 1 >= 0:
        neighbors.append((x - 1, y))
    else:
        neighbors.append((width - 1, y))

    # right cell
    if x + 1 < width:
        neighbors.append((x + 1, y))
    else:
        neighbors.append((0, y))

    # North cell
    if y - 1 >= 0:
        neighbors.append((x, y - 1))

    # south cell
    if y + 1 < height:
        neighbors.append((x, y + 1))

    return neighbors

In [ ]:

def get_next_site(x, y, height_map):
    neighbors = get_neighbors(x, y, height_map.shape[0], height_map.shape[1])

    delta_h = [height_map[x, y] - height_map[i, j] for i, j in neighbors]
    print("delta_h values:", delta_h)  # Debug line from net help

    exp_values = [beta * dh for dh in delta_h]
    print("exp_values:", exp_values) 

    weights = [np.exp(ev) if ev >= 0 else 0 for ev in exp_values]
    print("weights:", weights) 

    chosen_neighbor = random.choices(neighbors, weights)[0]

    # set limit to maximum 'depth'
    if height_map[chosen_neighbor] < height_map[x, y]:
        return chosen_neighbor
    else:
        return None

In [ ]:
def simulate_raindrop(x, y, height_map):
    path = [(x, y)]
    counter = 0
    max_iterations = 1000  #

    while y > 0:  # until the raindrop reaches bottom
        counter += 1
        if counter > max_iterations:
            #print(f"Exceeded {max_iterations} iterations at point ({x}, {y}).")
            #print("Path:", path)
            break  # countermeasure to not get stuck in local minimas
        next_site = get_next_site(x, y, height_map)
        if next_site is None:
            break  # We have reached a local minimum
        x, y = next_site
        path.append((x, y))

    return path

In [ ]:
paths = []  # List to hold all paths

for _ in tqdm(range(total_drops)):
    # Select a random starting point
    start_x = random.randint(0, grid_x - 1)
    start_y = random.randint(0, grid_y - 1)

    # Simulate the path of the raindrop
    path = simulate_raindrop(start_x, start_y, height_map)
    paths.append(path) # we save our path itself for later
    
    # Erode the height map along the path
    for x, y in path:
        height_map[x, y] -= Dh

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

X = np.arange(0, grid_x, 1)
Y = np.arange(0, grid_y, 1)
X, Y = np.meshgrid(X, Y)
Z = height_map[X, Y]

ax.plot_surface(X, Y, Z, cmap='terrain')

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Height')
plt.title('3D plot of eroded slope without avalanche correction')
plt.savefig('3D slope NoAvalanche.png')
plt.show()

In [ ]:
plt.figure(figsize=(10, 8))

plt.imshow(height_map, cmap='terrain', origin='lower')
plt.colorbar(label='Height')

plt.xlabel('X')
plt.ylabel('Y')
plt.title('2D plot of eroded slope without avalanche correction')

plt.show()

In [ ]:
river_network = np.zeros((grid_x, grid_y))

for path in paths:
    for x, y in path:
        river_network[x, y] += 1
# simple paths for rivers plot

# Plot the river network
plt.figure(figsize=(10, 8))
plt.imshow(river_network, origin='lower')
plt.xlabel('X')
plt.ylabel('Y')
plt.title('2D river_network')
plt.savefig('river_network NoAvalanche.png')
plt.show()

In [ ]:
def apply_avalanches(height_map, R):
    width, height = height_map.shape

    # Repeat until all Dh < R
    while True:
        Dh = np.zeros((width, height))
        for x in range(width):
            for y in range(height):
                neighbors = get_neighbors(x, y, width, height)
                for i, j in neighbors:
                    Dh[x, y] = max(Dh[x, y], height_map[x, y] - height_map[i, j])

        # If depthbecomes bigger than R, we reduce height by 0.25 * Dh
        if np.max(Dh) > R:
            for x in range(width):
                for y in range(height):
                    if Dh[x, y] > R:
                        height_map[x, y] -= 0.25 * Dh[x, y]
        else:
            break 

    return height_map

In [ ]:
Avalanche_height_map = apply_avalanches(height_map,190)

In [ ]:
paths_avalanche = []  # List to hold all paths

for _ in tqdm(range(total_drops)):
    # Select a random starting point
    start_x = random.randint(0, grid_x - 1)
    start_y = random.randint(0, grid_y - 1)

    # Simulate the path of the raindrop
    path_A = simulate_raindrop(start_x, start_y, Avalanche_height_map)
    paths_avalanche.append(path_A) # we save our path itself for later
    
    # Erode the height map along the path
    #for x, y in path:
    #    height_map[x, y] -= Dh